# **Predicción de la Deserción Estudiantil en la Universidad Nacional de Ingeniería (2018-2024) con Inteligencia Artificial** 

# **Lectura de los datos PreUniversitarios y Admisión**

In [340]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from unidecode import unidecode

In [341]:
data_admision = pd.read_csv(r'..\Datos_abiertos_admision_2021_1_2024_1.csv')
data_preuniversitario = pd.read_csv(r'..\Datos_abiertos_cepre.csv')

In [342]:
# data_preuniversitario.dtypes
data_admision.columns

Index(['IDHASH', 'COLEGIO', 'COLEGIO_DEPA', 'COLEGIO_PROV', 'COLEGIO_DIST',
       'COLEGIO_PAIS', 'COLEGIO_ANIO_EGRESO', 'ESPECIALIDAD', 'ANIO_POSTULA',
       'CICLO_POSTULA', 'DOMICILIO_DEPA', 'DOMICILIO_PROV', 'DOMICILIO_DIST',
       'ANIO_NACIMIENTO', 'NACIMIENTO_PAIS', 'NACIMIENTO_DEPA',
       'NACIMIENTO_PROV', 'NACIMIENTO_DIST', 'SEXO', 'CALIF_FINAL', 'INGRESO',
       'MODALIDAD'],
      dtype='object')

# **Objetivos**

### **1.1. Determinar el periodo significativo para integrar un modelo ML**

In [343]:
# Data CEPreUniversitario: 2016 - 2023
# Inscripción: Apartir del Lunes 08 de Julio hasta el 23 de Agosto
# Inicio de clases: 02 de Setiembre del 2024.
# Fin de clases: Diciembre del 2024

# Data Admision: 2021-1 al 2024-1
# Rendir 3 examenes de admisión
# Inicio en Febrero
# Inicio en Agosto

# Definir el alcance del periodo significativo para abordar un rango de analsis apropiado, lo cual sera desde 2020-1 hasta 2024-1
data_preuniversitario.query('ANIO_POSTULA >= 2020', inplace=True)
# data_preuniversitario

### **1.2. Determinar los estudiantes presentes en la PreUniversitario y Admisión**

### **1.2.1. Verificar y limpiar la data presente en las columnas**

In [344]:
# data_admision["IDHASH"].describe()
# data_preuniversitario["IDHASH"].describe()

# 1. Eliminar espacios en blancos que están presentes en la data
# data_admision["IDHASH"].str.strip().describe()
# data_preuniversitario["IDHASH"].str.strip().describe()

# 2. Verificar datos NAN o NULLS
# data_admision["IDHASH"].isnull().sum() // isna()
# data_preuniversitario["IDHASH"].isnull().sum() 

# 3. Detección de datos Duplicados (ROWS)
# data_admision.duplicated().sum()
# data_preuniversitario.duplicated().sum()

# IDHASH se descarta
# En conclusión, no se encontraron espacios rendundantes en el set Admin y PRE respecto al atributo IDHASH
# En conlusión, no se econtraron datos faltantes            ""
# En conclusión, no se encontraron datos dulpicados         ""

In [345]:
# 1. Eliminar espacios en blancos, letras en mayusculas y eliminar acentos que están presentes en la data

# Identificar las columnas de tipo 'object'
object_cols_1 = data_admision.select_dtypes(include='object').columns
object_cols_2 = data_preuniversitario.select_dtypes(include='object').columns

# Función para limpiar texto, manejando posibles NaN u otros tipos de datos
def clean_column(column):
    return column.str.strip().str.upper().map(lambda x: unidecode(x) if isinstance(x, str) else x)

# Aplicar la limpieza a cada columna de tipo 'object'
for col in object_cols_1:
    data_admision[col] = clean_column(data_admision[col])
    
for col in object_cols_2:
    data_preuniversitario[col] = clean_column(data_preuniversitario[col])

# Contar valores únicos de las columnas
# data_admision.dtypes()

### **1.2.2. Imputación de datos faltantes**

In [348]:
# 2. Identificar columnas que presentan NAN o NULLS
data_admision.isnull().sum()[data_admision.isnull().sum() > 0]

COLEGIO_DEPA         226
COLEGIO_PROV         228
COLEGIO_DIST         228
COLEGIO_PAIS       23574
ESPECIALIDAD          55
NACIMIENTO_DEPA      202
NACIMIENTO_PROV      202
NACIMIENTO_DIST      203
dtype: int64

In [347]:
# 2.1. Imputación de datos de Calificación 
# data_admision["CALIF_FINAL"].info() // Distribución asimetrica positiva
# data_admision["CALIF_FINAL"][data_admision["CALIF_FINAL"].isnull()]
data_admision['CALIF_FINAL'].fillna(data_admision['CALIF_FINAL'].mode()[0], inplace=True)
data_admision["CALIF_FINAL"].isnull().count()

42516

In [354]:
# 2.2 Imputacion de datos de Colegio pais
data = data_admision[data_admision["COLEGIO"].value_counts()<100]


C:\Users\user\AppData\Local\Temp\ipykernel_13452\2488097030.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data_admision[data_admision["COLEGIO"].value_counts()<100]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [326]:
data_admision.groupby(data_admision["NACIMIENTO_PAIS"]).size()

NACIMIENTO_PAIS
ARGENTINA           53
BOLIVIA              9
BRAZIL               9
CHILE               16
COLOMBIA             6
ECUADOR              4
EE.UU.              17
ESPANA              16
FRANCIA              1
GUATEMALA            2
HONDURAS             1
ITALIA              19
JAPON               22
KOREA                2
MEXICO               1
NO ESPECIFICA        2
PARAGUAY             1
PERU             42285
SIRIA                1
SUIZA                4
URUGUAY              1
VENEZUELA           44
dtype: int64

#### **1.2.2. Identificar estudiantes presentes en la PreUniversitario y Admision**

In [180]:
# Mediante un Query de Python podemos identificar aquellos estudiantes presentes en PreUniversitario y Admision.
# Inner JOIN. 
merged_inner = pd.merge(left=data_preuniversitario, right=data_admision, how='inner', left_on='IDHASH', right_on='IDHASH')
merged_inner = merged_inner[['IDHASH']]
merged_inner.describe()

# merged_inner.unique()

,IDHASH
count,32216
unique,7430
top,74831DDBBB0B69C9CC1D3E6DBA61310C3AB2B26A94ABF5...
freq,49
